In [1]:
"""
libreria requerida para la el proceso ETL 
"""
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os

"""Generacion de variables y Folders en caso de queno exitan
un vez generado el folder sera necesario cargar el archivo CSV 
a nuestra folder RAW
"""
RAW_FOLDER='./raw/'
PROCESSED_FOLDER='./processed/'
RAW_FILE='data_prueba_tecnica.csv'
PROCESSED_FILE='data_processed.csv'
PROCESSES_VALIDATION_FILE='data_validation.csv'
SEP=','
ENCODING='utf-8'

os.makedirs(RAW_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

In [3]:
def read_raw_data(path_file, file_name, sep, econding):
    """Lectura de la archivo de capa raw 
    Args:
        path_file (string): Directorio del archivo
        file_name (string): Nombre del archivo
        sep (string) : Separador de columnas de nuestro archivo
        encoding (string) : Codoficacion del archivo
    Returns:
        dataframe: Dataframe con los valores de raw
    """
    df = pd.read_csv(path_file+file_name, sep=sep, encoding=econding)
    return df

In [14]:
def clean_data(df):
    """Limpieza y tranformacion del data frame generdo en la capa raw de lectura,
    esta funcion elimina duplicados, nulos. transfroma los campos creat_at y paid_at a tipo fecha
    ademas de generar como reaultado 2 data frames.
    Args:
        df(dataframe): datfreme de lectira de capa raw 
    Returns:
        df_clean: Datafreme tranformado y limpio
        df_inc:  Dtafrem con valores inconsistentes
    """
    
    # print(df.columns)
    # print(df.dtypes)
    # nulos_por_columna = df.isnull().sum()
    # print(nulos_por_columna)
    # print(df.info)
    df_clean = df.drop_duplicates()
    df_clean['created_at'] = pd.to_datetime(
        df_clean['created_at'], errors='coerce')
    df_clean['paid_at'] = pd.to_datetime(df_clean['paid_at'], errors='coerce')
    df_inc = df[df_clean[['created_at', 'id',
                          'name', 'company_id']].isnull().any(axis=1)]
    df_clean = df_clean.dropna(
        subset=['created_at', 'id', 'name', 'company_id'])

    # print(df_clean)

    return df_clean, df_inc

In [15]:
def save_data(df, file_path, file_name, sep, encoding):
    """Guardado de un archivo en capa processed a partir de un dataframe 

    Args:
        df (dataframe): Data a guardar 
        path_file (string): Directorio del archivo
        file_name (string): Nombre del archivo
        sep (string) : Separador de columnas de nuestro archivo
        encoding (string) : Codoficacion del archivo
    Returns:
        dataframe: Dataframe con los valores de raw
    """
    df.to_csv(file_path+file_name, sep=sep, encoding=encoding, index=False)

In [16]:
def ventas_dia(df):
    """"
    Generacion de agraciones sobre los campos name ,
    creare_at renombrado date y amount renombrado total_amount 
    Args:
        df(dataframe): Datfreme transformdo y limpio 
    Returns:
        df (dataframe): Datafreme con agregacion a nivel dia
    """
    df = df.groupby(['name', df['created_at'].dt.date])[
        'amount'].sum().reset_index()
    df.rename(columns={'created_at': 'date',
                    'amount': 'total_amount'}, inplace=True)
    return df

In [50]:
def patron_pago(df):
    """"
    Generacion de agraciones sobre los campos name ,
    creare_at renombrado date y amount renombrado total_amount 
    Args:
        df(dataframe): Datfreme transformdo y limpio
    Returns:
        df_clean: Datafreme tranformado y limpio
    """
    df_pagado = df[df['status'] == 'paid'].copy()

    # 2. Calcular el tiempo de pago (solo para las transacciones filtradas)
    df_pagado['tiempo_pago'] = (df_pagado['paid_at'] - df_pagado['created_at']).dt.days

    # 3. Calcular la desviación estándar del tiempo de pago por empresa
    # regularidad = df_pagado.groupby('company_id')['tiempo_pago'].std().reset_index()
    regularidad = df_pagado.groupby('company_id')['tiempo_pago'].std().reset_index()
    regularidad.rename(columns={'tiempo_pago': 'desviacion_estandar'}, inplace=True)

    # 4. Clasificación según la desviación estándar
    def clasificar_regularidad(std):
        if std < 10:
            return "Alta Regularidad"
        elif 10 <= std <= 30:
            return "Moderada Regularidad"
        else:
            return "Irregular"

    regularidad['clasificacion'] = regularidad['desviacion_estandar'].apply(clasificar_regularidad)

    # Resultados
    # print("DataFrame filtrado (status = 'pagado'):")
    # print(df_pagado)

    print("\nRegularidad de las empresas:")
    print(regularidad)

In [51]:
df_raw = read_raw_data(RAW_FOLDER,
                       RAW_FILE, SEP, ENCODING)

df_clean, df_inc = clean_data(df_raw)
df_clean_agg=ventas_dia(df_clean)
df_patron_pago=patron_pago(df_clean)
save_data(df_clean_agg, PROCESSED_FOLDER,
          PROCESSED_FILE, ',', 'utf-8')
save_data(df_inc, PROCESSED_FOLDER,
          'data_validation.csv', ',', 'utf-8')


Regularidad de las empresas:
                                 company_id  desviacion_estandar  \
0                                   *******                  NaN   
1  8f642dc67fccf861548dfe1c761ce22f795e91f0             0.678911   
2  cbf1c8b09cd5b549416d49d220a40cbd317f952e             0.248195   

      clasificacion  
0         Irregular  
1  Alta Regularidad  
2  Alta Regularidad  
